In [2]:
import pandas as pd

In [17]:
df = pd.read_excel('MDCR SUMMARY AB_CPS11SAB_2023_Copy.xlsx', 
                   sheet_name='MDCR SUMMARY AB 5_CPS_11SAB',
                   header=None)

print(df.shape)
df.head(10)

(73, 20)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,NaN,Medicare Part A and Part B Summary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"Utilization, Program Payments, and Cost Sharin...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,"Area of Residence, Calendar Year 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Area of Residence,Total Original Medicare Part A and/or Part B E...,Total Medicare Part A and/or Part B Persons Wi...,Total Medicare Part A and/or Part B Program Pa...,Medicare Part A and/or Part B Program Payments...,Medicare Part A and/or Part B Program Payments...,Total Original Medicare Part A Enrollees,Total Medicare Part A Persons With Utilization,Total Medicare Part A Program Payments,Medicare Part A Program Payments Per Person Wi...,Medicare Part A Program Payments Per Original ...,Total Original Medicare Part B Enrollees,Total Medicare Part B Persons With Utilization,Total Medicare Part B Program Payments,Medicare Part B Program Payments Per Person Wi...,Medicare Part B Program Payments Per Original ...,Total Beneficiary Cost Sharing,Beneficiary Cost Sharing Per Person With Utili...,Beneficiary Cost Sharing Per Original Medicare...,Discharged Dead
4,All Areas,34367703,33727368,405881283931,12034,11810,34059184,6331939,185764927491,29338,5454,28623931,32960714,220116356440,6678,7690,68797265225,2040,2002,1569164
5,United States,33694472,33583717,405095202739,12062,12023,33386787,6303598,185262168601,29390,5549,28516856,32835526,219833034138,6695,7709,68682469508,2045,2038,1560222
6,Alabama,471490,487147,5339582525,10961,11325,465089,106150,2436050598,22949,5238,395095,470119,2903531927,6176,7349,895894183,1839,1900,29099
7,Alaska,111904,94497,1186174763,12553,10600,111363,13872,495993530,35755,4454,98579,93604,690181233,7373,7001,211706097,2240,1892,2044
8,Arizona,745551,753092,8952374023,11887,12008,735576,132048,3535126514,26772,4806,635802,732766,5417247508,7393,8520,1560200438,2072,2093,35489
9,Arkansas,382179,379230,4227322195,11147,11061,381828,74515,1925230353,25837,5042,335347,371825,2302091842,6191,6865,756672264,1995,1980,18117


In [18]:
# Set row 3 as the header
df.columns = df.iloc[3]

# Keep only rows from row 5 onward (actual data)
df = df.iloc[5:].reset_index(drop=True)

# Drop rows where Area of Residence is NaN or 'BLANK'
df = df.dropna(subset=['Area of Residence'])
df = df[df['Area of Residence'] != 'BLANK']

# Keep only the 50 states + DC (rows 0-51 after cleaning)
# Remove territories, foreign countries, unknowns
exclude = ['All Areas', 'United States', 'Puerto Rico', 'Virgin Islands', 
           'American Samoa', 'Guam', 'Northern Mariana Islands', 
           'Foreign Countries', 'Unknown', 
           'Territories, Possessions, and Other']
df = df[~df['Area of Residence'].isin(exclude)]

# Reset index
df = df.reset_index(drop=True)

# Check what we have
print(df.shape)
print(df['Area of Residence'].tolist())

(51, 20)
['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']


In [22]:
# Rename columns to something clean and SQL-friendly
df_states = df[['Area of Residence',
                'Total Original Medicare Part A and/or Part B Enrollees',
                'Total Medicare Part A and/or Part B Program Payments',
                'Medicare Part A and/or Part B Program Payments Per Original Medicare Enrollee',
                'Total Medicare Part A Program Payments',
                'Total Medicare Part B Program Payments',
                'Total Beneficiary Cost Sharing']].copy()

df_states.columns = [
    'State',
    'Total_enrollees',
    'Total_program_payments',
    'Payments_per_enrollee',
    'Part_a_payments',
    'Part_b_payments',
    'Beneficiary_cost_sharing'
]

# Convert numeric columns to numbers (they may be stored as objects)
numeric_cols = ['Total_enrollees', 'Total_program_payments', 'Payments_per_enrollee',
                'Part_a_payments', 'Part_b_payments', 'Beneficiary_cost_sharing']
df_states[numeric_cols] = df_states[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Preview
print(df_states.shape)
df_states.head()

(51, 7)


,State,Total_enrollees,Total_program_payments,Payments_per_enrollee,Part_a_payments,Part_b_payments,Beneficiary_cost_sharing
0,Alabama,471490,5339582525,11325,2436050598,2903531927,895894183
1,Alaska,111904,1186174763,10600,495993530,690181233,211706097
2,Arizona,745551,8952374023,12008,3535126514,5417247508,1560200438
3,Arkansas,382179,4227322195,11061,1925230353,2302091842,756672264
4,California,3401066,47902545961,14085,23672209013,24230336948,6965749443


In [26]:
# Save to output folder
df_states.to_csv('C:/Users/16503/Downloads/medicare-eda-project/output/ab5_states_2023.csv', index=False)
print("Saved successfully!")

Saved successfully!


In [29]:
# Load AB4
df4 = pd.read_excel('MDCR SUMMARY AB_CPS11SAB_2023_Copy.xlsx', 
                    sheet_name='MDCR SUMMARY AB 4_CPS_11SAB',
                    header=None)

# Set headers and slice data
df4.columns = df4.iloc[3]
df4 = df4.iloc[5:].reset_index(drop=True)

# Check column names
print(df4.columns.tolist())

['Demographic Characteristic', 'Total Original Medicare Part A and/or Part B Enrollees', 'Total Medicare Part A and/or Part B Persons With Utilization', 'Total Medicare Part A and/or Part B Program Payments', '% of Total Medicare Part A and/or Part B Program Payments', 'Medicare Part A and/or Part B Program Payments Per Person With Utilization', 'Medicare Part A and/or Part B Program Payments Per Original Medicare Enrollee', 'Total Original Medicare Part A Enrollees', 'Total Medicare Part A Persons With Utilization', 'Total Medicare Part A Program Payments', '% of Total Medicare Part A Program Payments', 'Medicare Part A Program Payments Per Person With Utilization', 'Medicare Part A Program Payments Per Original Medicare Enrollee', 'Total Original Medicare Part B Enrollees', 'Total Medicare Part B Persons With Utilization', 'Total Medicare Part B Program Payments', '% of Total Medicare Part B Program Payments', 'Medicare Part B Program Payments Per Person With Utilization', 'Medicare 

In [31]:
# Define the demographic rows we want to keep
keep_rows = [
    'Under 65 Years', '65 Years and Over',
    'Under 18 Years', '18-24 Years', '25-34 Years', '35-44 Years',
    '45-54 Years', '55-64 Years', '65-74 Years', '75-84 Years',
    '85-94 Years', '95 Years and Over',
    'Male', 'Female',
    'Non-Hispanic White', 'Black (or African-American)',
    'Asian/Pacific Islander', 'Hispanic',
    'American Indian/Alaska Native',
    'MME', 'Non-MME'
]

df_demo = df4[df4['Demographic Characteristic'].isin(keep_rows)].copy()

# Keep only the most useful columns
df_demo = df_demo[['Demographic Characteristic',
                   'Total Original Medicare Part A and/or Part B Enrollees',
                   'Total Medicare Part A and/or Part B Program Payments',
                   'Medicare Part A and/or Part B Program Payments Per Original Medicare Enrollee',
                   'Total Beneficiary Cost Sharing ',
                   'Beneficiary Cost Sharing Per Original Medicare Enrollee']].copy()

# Rename columns
df_demo.columns = [
    'demographic_group',
    'total_enrollees',
    'total_program_payments',
    'payments_per_enrollee',
    'total_cost_sharing',
    'cost_sharing_per_enrollee'
]

# Convert to numeric
numeric_cols = ['total_enrollees', 'total_program_payments', 'payments_per_enrollee',
                'total_cost_sharing', 'cost_sharing_per_enrollee']
df_demo[numeric_cols] = df_demo[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Reset index and preview
df_demo = df_demo.reset_index(drop=True)
print(df_demo.shape)
df_demo.head(10)

(21, 6)


,demographic_group,total_enrollees,total_program_payments,payments_per_enrollee,total_cost_sharing,cost_sharing_per_enrollee
0,Under 65 Years,3533771,45123466717,12769,7874163260,2228
1,65 Years and Over,30833932,360757817215,11700,60923101966,1976
2,Under 18 Years,1178,70676902,60014,8631624,7329
3,18-24 Years,58752,390397151,6645,74499133,1268
4,25-34 Years,313894,2852696965,9088,524122038,1670
5,35-44 Years,569149,6243165437,10969,1128756599,1983
6,45-54 Years,852996,10729358523,12578,1904408557,2233
7,55-64 Years,1737802,24837171738,14292,4233745308,2436
8,65-74 Years,17391872,144815447473,8327,26726905680,1537
9,75-84 Years,9754678,137860183571,14133,23270422980,2386


In [35]:
df_demo.to_csv('C:/Users/16503/Downloads/medicare-eda-project/output/ab4_demographics_2023.csv', index=False)
print("Saved successfully!")

Saved successfully!


In [39]:
# Load AB1
df1 = pd.read_excel('MDCR SUMMARY AB_CPS11SAB_2023_Copy.xlsx', 
                    sheet_name='MDCR SUMMARY AB 1_CPS_11SAB',
                    header=None)

# Row 3 has the headers (years), column 0 has service categories
df1.columns = ['service_type', 2018, 2019, 2020, 2021, 2022, 2023]

# Keep only the Program Payments rows — rows 35 to 44
df_trends = df1.iloc[35:45].copy()

# Preview
print(df_trends)

                                service_type          2018          2019  \
35                      Part A and/or Part B  3.895257e+11  4.002402e+11   
36                                    Part A  1.907015e+11  1.932513e+11   
37               Inpatient Hospital Services  1.369960e+11  1.385252e+11   
38         Skilled Nursing Facility Services  2.765224e+10  2.713184e+10   
39                          Hospice Services  1.925103e+10  2.089897e+10   
40               Home Health Agency Services  6.802182e+09  6.695304e+09   
41                                    Part B  1.988242e+11  2.069889e+11   
42      Physician and Other Medical Services  1.099281e+11  1.148125e+11   
43                       Outpatient Services  7.776420e+10  8.102076e+10   
44               Home Health Agency Services  1.113187e+10  1.115556e+10   

            2020          2021          2022          2023  
35  3.796014e+11  3.961222e+11  3.950043e+11  4.058813e+11  
36  1.874457e+11  1.877131e+11  1.863510e

In [42]:
# Rename the service type column values to clean names
df_trends = df_trends.reset_index(drop=True)

# Assign clean service type names based on what we know the rows are
df_trends['service_type'] = [
    'Total Part A and B',
    'Part A Total',
    'Inpatient Hospital',
    'Skilled Nursing Facility',
    'Hospice',
    'Home Health Agency (Part A)',
    'Part B Total',
    'Physician and Other Medical',
    'Outpatient Services',
    'Home Health Agency (Part B)'
]

# Convert year columns to numeric
year_cols = [2018, 2019, 2020, 2021, 2022, 2023]
df_trends[year_cols] = df_trends[year_cols].apply(pd.to_numeric, errors='coerce')

# Preview
df_trends

,service_type,2018,2019,2020,2021,2022,2023
0,Total Part A and B,3.895257e+11,4.002402e+11,3.796014e+11,3.961222e+11,3.950043e+11,4.058813e+11
1,Part A Total,1.907015e+11,1.932513e+11,1.874457e+11,1.877131e+11,1.863510e+11,1.857649e+11
2,Inpatient Hospital,1.369960e+11,1.385252e+11,1.301575e+11,1.313175e+11,1.277373e+11,1.269471e+11
3,Skilled Nursing Facility,2.765224e+10,2.713184e+10,2.855455e+10,2.722280e+10,2.900932e+10,2.727573e+10
4,Hospice,1.925103e+10,2.089897e+10,2.244535e+10,2.306571e+10,2.369820e+10,2.566940e+10
5,Home Health Agency (Part A),6.802182e+09,6.695304e+09,6.288322e+09,6.107150e+09,5.906163e+09,5.872733e+09
6,Part B Total,1.988242e+11,2.069889e+11,1.921557e+11,2.084091e+11,2.086533e+11,2.201164e+11
7,Physician and Other Medical,1.099281e+11,1.148125e+11,1.065491e+11,1.173906e+11,1.164130e+11,1.263432e+11
8,Outpatient Services,7.776420e+10,8.102076e+10,7.481262e+10,8.025276e+10,8.198538e+10,8.378105e+10
9,Home Health Agency (Part B),1.113187e+10,1.115556e+10,1.079401e+10,1.076568e+10,1.025496e+10,9.992150e+09


In [46]:
# Melt from wide to long format
df_trends_long = df_trends.melt(
    id_vars='service_type',
    value_vars=[2018, 2019, 2020, 2021, 2022, 2023],
    var_name='year',
    value_name='program_payments'
)

# Save to CSV
df_trends_long.to_csv('C:/Users/16503/Downloads/medicare-eda-project/output/ab1_trends_2018_2023.csv', index=False)
print(df_trends_long.shape)
df_trends_long.head(20)

(60, 3)


,service_type,year,program_payments
0,Total Part A and B,2018,3.895257e+11
1,Part A Total,2018,1.907015e+11
2,Inpatient Hospital,2018,1.369960e+11
3,Skilled Nursing Facility,2018,2.765224e+10
4,Hospice,2018,1.925103e+10
5,Home Health Agency (Part A),2018,6.802182e+09
6,Part B Total,2018,1.988242e+11
7,Physician and Other Medical,2018,1.099281e+11
8,Outpatient Services,2018,7.776420e+10
9,Home Health Agency (Part B),2018,1.113187e+10


# Medicare EDA Project — Data Cleaning

## Data Source
CMS Medicare Part A and Part B Summary, 2023
https://data.cms.gov/summary-statistics-on-use-and-payments/medicare-medicaid-service-type-reports/cms-program-statistics-medicare-part-a-part-b-all-types-of-service

## What this notebook does
- Loads raw CMS Medicare data from Excel
- Cleans and extracts three analysis-ready datasets:
  - **ab5_states_2023.csv** - Program payments by US state (51 rows)
  - **ab4_demographics_2023.csv** - Program payments by demographic group (21 rows)
  - **ab1_trends_2018_2023.csv** - Program payments by service type, 2018–2023 (60 rows)

## Key decisions
- Excluded territories, foreign countries, and unknown geographies from state data
- Reshaped AB1 from wide to long format for easier analysis
- Converted all payment columns to numeric